In [1]:
# addprocs(7)  # parallel computing

In [2]:
include("prepare_data_Sioux_simp.jl");
include("extract_data.jl");
include("tap_MSA_Sioux_simp.jl");
include("demands_adjustment_Sioux_simp_gradi.jl");

In [3]:
fcoeffs = [1, 0, 0, 0, .15];

In [4]:
tapFlowVecDict = Dict{Int64, Array{Float64, 1}}();
tapFlowDicDict = Dict{Int64, Dict{(Int64,Int64),Float64}}();

In [5]:
# get observed flow vector (corresponding to ground truth demands)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [6]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], fcoeffs);

In [7]:
jacobDict = Dict{Int64, Array{Float64}}();

jacobDict[1] = jacobian(tapFlowVecDict[1], fcoeffs, capacity, free_flow_time, numLinks, numODpairs, 
numRoutes, linkRoute, odPairRoute);

In [8]:
jacobDict;

In [9]:
descDirecDict = Dict{Int64, Array{Float64,1}}();
descDirecDict[1] = descDirec(tapFlowVecDict[1], tapFlowVecDict[0], jacobDict[1], numODpairs, numLinks);

In [10]:
demandsVecDict = Dict{Int64, Array{Float64,1}}();
demandsVecDict[0] = demandsDicToVec(demandsDict[0]);
demandsVecDict[1] = demandsDicToVec(demandsDict[1]);

In [11]:
searchDirecDict = Dict{Int64, Array{Float64,1}}();
searchDirecDict[1] = searchDirec(demandsVecDict[1], descDirecDict[1], 20);

In [12]:
thetaMaxDict = Dict{Int64, Float64}();
thetaMaxDict[1] = thetaMax(demandsVecDict[1], searchDirecDict[1]);

In [13]:
objFunDict = Dict{Int64, Float64}();
objFunDict[1] = objF(demandsVecDict[1]);

In [14]:
demandsVecDict[2] = similar(demandsVecDict[0]);
demandsVecDict[2], objFunDict[2] = armijo(demandsVecDict[1], searchDirecDict[1], thetaMaxDict[1], 2, 10);

In [15]:
demandsVecDict

Dict{Int64,Array{Float64,1}} with 3 entries:
  0 => [100.0,100.0,500.0,200.0,300.0,500.0,800.0,500.0,1300.0,500.0,200.0,100.0…
  2 => [92.1524415098556,99.57852133783419,493.82913760219856,190.2551932127423,…
  1 => [94.66405926995482,98.67525166987629,503.9914657537483,188.2389715867282,…

In [16]:
objFunDict

Dict{Int64,Float64} with 2 entries:
  2 => 308910.85799380176
  1 => 359156.1146109561